<a href="https://colab.research.google.com/github/justadudewhohacks/ipynbs/blob/master/face_detection_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [0]:
!pip install -U -q PyDrive
!pip install git+https://github.com/justadudewhohacks/image_augment.py
!pip install git+https://github.com/justadudewhohacks/colabsnippets

## mxnet

In [0]:
!nvcc --version
!pip install mxnet-cu100

## pytorch

In [0]:
!pip install torch torchvision

# Download Data

In [3]:
from colabsnippets.DataDownloader import DataDownloader

data_downloader = DataDownloader(data_dir = './data')

data_downloader.download_data({
	"celeba" : [
    #test data starts at shard 18
    { "images": "1C19zO2dlQz4ShGFJH_zEi3ToDOH-Iomp" },
    { "images": "1S5IIoHbNjv3p7PkJ3ItoAeQJODBX597-" },
    { "images": "1e6TUSjz7Zv6LmDP69Yc3lRIufeK3FuAR" }
	]
}, [])
data_downloader.drive.CreateFile({ 'id': '1rujtZzpWX5DP7E7HaxgqOPpk0cRb0ajD' }).GetContentFile('./data/celeba/landmarks.json')
data_downloader.download_data({
	"WIDER" : [
    { "images": "1JHmXqGPngDCbM56eYPeqsaCgJC4vgL4m", "boxes": "1Hd2i-6dnaWIriFK4Hj0CLZnfGtKcKj9L" }
	]
}, ['boxes'])
data_downloader.download_data({
	"FDDB" : [
    { "images": "1C8RpAZYg5nsPCAOLESGFwKxEaHx5V3Ag", "boxes": "1ACZPuSB7j0c0_hDIBL_MSW4rrzPhC1ab" }
	]
}, ['boxes'])

print('done!')

W0620 05:52:40.180226 140364883408768 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

downloading data for db: celeba
downloading data for shard 0
unzipping images.7z done in 41.83179688453674s
downloading data for shard 1
unzipping images.7z done in 40.39577627182007s
downloading data for shard 2
unzipping images.7z done in 7.8820226192474365s
download_data - total time: 107.1499376296997s
downloading data for db: WIDER
downloading data for shard 0
unzipping images.7z done in 127.27363204956055s
unzipping boxes.7z done in 51.95311212539673s
download_data - total time: 207.77921199798584s
downloading data for db: FDDB
downloading data for shard 0
unzipping images.7z done in 5.044902324676514s
unzipping boxes.7z done in 1.5364875793457031s
download_data - total time: 8.680900573730469s
done!


# DataLoader

In [0]:
import cv2
import math
import json
import random
import time
import types
import os
import numpy as np
import tensorflow as tf
from augment import ImageAugmentor, augment
from augment.augment import abs_coords
from colabsnippets.utils import load_json
from colabsnippets import BatchLoader

'''
--------------------------------------------------------------------------------

Data Loader

--------------------------------------------------------------------------------
'''
celeba_landmarks_by_file = load_json('./data/celeba/landmarks.json')

def min_bbox_from_pts(pts):
  min_x, min_y, max_x, max_y = 1.0, 1.0, 0, 0
  for pt in pts:
    x, y = pt
    min_x = x if x < min_x else min_x
    min_y = y if y < min_y else min_y
    max_x = max_x if x < max_x else x
    max_y = max_y if y < max_y else y

  return [min_x, min_y, max_x, max_y]

def min_bbox(boxes):
  min_x, min_y, max_x, max_y = 1.0, 1.0, 0, 0
  for box in boxes:
    x, y, w, h = box
    pts = [(x, y), (x + w, y + h)]
    for x, y in pts:
      min_x = x if x < min_x else min_x
      min_y = y if y < min_y else min_y
      max_x = max_x if x < max_x else x
      max_y = max_y if y < max_y else y

  return [min_x, min_y, max_x, max_y]
 
def json_boxes_to_array(boxes):
  out_boxes = []
  for box in boxes:
    x, y, w, h = box['x'], box['y'], box['width'], box['height']
    out_box = (x, y, w, h)     
    if w <= 0 or h <= 0:
      raise Exception("box has invalid width or height: {}".format(out_box))   
    for val in out_box:
      if val < -0.5 or val > 1.5:
        raise Exception("box is probably not a valid relative box: {}".format(out_box))
    out_boxes.append(out_box)
  return out_boxes
 
def extract_data_labels(data):
  db = data['db']
  img_file = data['file']
  if db == 'celeba':
    landmarks = celeba_landmarks_by_file[img_file]
    x, y, max_x, max_y = min_bbox_from_pts(landmarks)
    w, h = max_x - x, max_y - y
    padding = 1.5

    x = x - (0.5 * padding * w)
    y = y - (0.5 * padding * h)
    w = w + (padding * w)
    h = h + (padding * h)

    return [(x, y, w, h)]
  if db == 'WIDER' or db == 'FDDB':
    boxes_file = img_file.replace('.jpg', '.json')
    boxes_dir = "boxes-shard{}".format(data['shard']) if 'shard' in data else 'boxes'
    boxes_path = "./data/{}/{}/{}".format(db, boxes_dir, boxes_file)
    boxes = load_json(boxes_path)
    return json_boxes_to_array(boxes)
  
  raise Exception("extract_data_labels - unknown db '{}'".format(db))
    
    
def resolve_image_path(data):
  db = data['db']
  img_file = data['file']
  img_dir = "images-shard{}".format(data['shard']) if 'shard' in data else 'images'
  img_path = "./data/{}/{}/{}".format(db, img_dir, img_file)
  return img_path

class DataLoader(BatchLoader):
  def __init__(self, data, image_augmentor = None, is_test = False):  
    self.image_augmentor = image_augmentor
    BatchLoader.__init__(
      self, 
      data if type(data) is types.FunctionType else lambda: data, 
      resolve_image_path, 
      extract_data_labels,
      start_epoch = None, 
      is_test = is_test
    )
      
  def load_image_and_labels_batch(self, datas, image_size):
    batch_x, batch_y = [], []
    for data in datas:
      image = self.load_image(data)
      boxes = self.extract_data_labels(data)
      if self.image_augmentor is not None:
        roi = min_bbox(boxes)
        image, boxes = self.image_augmentor.augment(image, boxes = boxes, random_crop = roi, pad_to_square = True, resize = image_size)
      else:
        image, boxes = augment(image, boxes = boxes, pad_to_square = True, resize = image_size)
      batch_x.append(image)
      batch_y.append(boxes)
        
    return batch_x, batch_y
      

# Retina Face
https://github.com/deepinsight/insightface

In [0]:
!git clone https://github.com/deepinsight/insightface
!cd ./insightface/RetinaFace && make
!wget https://www.dropbox.com/s/53ftnlarhyrpkg2/retinaface-R50.zip -O retinaface-R50.zip
!unzip retinaface-R50.zip

In [0]:
import cv2
import sys
import numpy as np
import datetime
import os
import glob
import sys
sys.path.append('./insightface/RetinaFace')
from insightface.RetinaFace.retinaface import RetinaFace

def create_retina_face(scales = [1.0], do_flip = False):
  detector = RetinaFace('./R50', 0, 0, 'net3')
  def detect(img, min_score):
    faces, landmarks = detector.detect(img, min_score, scales=scales, do_flip=do_flip)
    out_faces = []
    img_size = img.shape[0]
    if faces is not None:
      for face in faces:
        x0, y0, x1, y1 = face.astype(np.int)[0:4] / img_size
        out_faces.append([x0, y0, x1-x0, y1-y0])
    return out_faces
  return detect

# DSFD
https://github.com/TencentYoutuResearch/FaceDetection-DSFD

In [0]:
!git clone https://github.com/TencentYoutuResearch/FaceDetection-DSFD
DataDownloader().drive.CreateFile({ 'id': '1WeXlNYsM6dMP3xQQELI-4gxhwKUQxc3-' }).GetContentFile('./WIDERFace_DSFD_RES152.pth')

In [0]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
import sys
sys.path.append('/content/FaceDetection-DSFD')

from face_ssd import build_ssd

def test_base_transform(image, mean):
    x = image.astype(np.float32)
    x -= mean
    x = x.astype(np.float32)
    return x


class TestBaseTransform:
    def __init__(self, mean):
        self.mean = np.array(mean, dtype=np.float32)

    def __call__(self, image, boxes=None, labels=None):
        return test_base_transform(image, self.mean), boxes, labels

torch.set_default_tensor_type('torch.cuda.FloatTensor')

def bbox_vote(det):
  order = det[:, 4].ravel().argsort()[::-1]
  det = det[order, :]
  while det.shape[0] > 0:
    # IOU
    area = (det[:, 2] - det[:, 0] + 1) * (det[:, 3] - det[:, 1] + 1)
    xx1 = np.maximum(det[0, 0], det[:, 0])
    yy1 = np.maximum(det[0, 1], det[:, 1])
    xx2 = np.minimum(det[0, 2], det[:, 2])
    yy2 = np.minimum(det[0, 3], det[:, 3])
    w = np.maximum(0.0, xx2 - xx1 + 1)
    h = np.maximum(0.0, yy2 - yy1 + 1)
    inter = w * h
    o = inter / (area[0] + area[:] - inter)
    # get needed merge det and delete these det
    merge_index = np.where(o >= 0.3)[0]
    det_accu = det[merge_index, :]
    det = np.delete(det, merge_index, 0)
    if merge_index.shape[0] <= 1:
      continue
    det_accu[:, 0:4] = det_accu[:, 0:4] * np.tile(det_accu[:, -1:], (1, 4))
    max_score = np.max(det_accu[:, 4])
    if type(max_score) == torch.Tensor:
      max_score = np.array(max_score.cpu())
      
    det_accu_sum = np.zeros((1, 5))
    s1 = np.sum(det_accu[:, -1:])
    if type(s1) == torch.Tensor:
      s1 = np.array(s1.cpu())
    det_accu_sum[:, 0:4] = np.sum(det_accu[:, 0:4], axis=0) / s1
    det_accu_sum[:, 4] = max_score
    try:
        dets = np.row_stack((dets, det_accu_sum))
    except:
        dets = det_accu_sum
  dets = dets[0:750, :]
  return dets
  
def infer(net , img , transform , thresh , shrink):
  if shrink != 1:
    img = cv2.resize(img, None, None, fx=shrink, fy=shrink, interpolation=cv2.INTER_LINEAR)
    
    
  x = torch.from_numpy(transform(img)[0]).permute(2, 0, 1)
  x = Variable(x.unsqueeze(0) , volatile=True)
  x = x.cuda()
  #print (shrink , x.shape)
  y = net(x)
  detections = y.data
  # scale each detection back up to the image
  scale = torch.Tensor([ img.shape[1]/shrink, img.shape[0]/shrink,
                       img.shape[1]/shrink, img.shape[0]/shrink] )
  det = []
  for i in range(detections.size(1)):
    j = 0
    while detections[0, i, j, 0] >= thresh:
      score = detections[0, i, j, 0]
      #label_name = labelmap[i-1]
      pt = (detections[0, i, j, 1:]*scale).cpu().numpy()
      coords = (pt[0], pt[1], pt[2], pt[3]) 
      det.append([pt[0], pt[1], pt[2], pt[3], score])
      j += 1
  if (len(det)) == 0:
    det = [ [0.1,0.1,0.2,0.2,0.01] ]
  det = np.array(det)

  keep_index = np.where(det[:, 4] >= 0)[0]
  det = det[keep_index, :]
  return det

def infer_flip(net , img , transform , thresh , shrink):
  img = cv2.flip(img, 1)
  det = infer(net , img , transform , thresh , shrink)
  det_t = np.zeros(det.shape)
  det_t[:, 0] = img.shape[1] - det[:, 2]
  det_t[:, 1] = det[:, 1]
  det_t[:, 2] = img.shape[1] - det[:, 0]
  det_t[:, 3] = det[:, 3]
  det_t[:, 4] = det[:, 4]
  return det_t
  
def create_dsfd(max_size = 608):
  net = build_ssd('test', 640, 2)
  net.load_state_dict(torch.load('./WIDERFace_DSFD_RES152.pth'))
  net.cuda()
  net.eval()

  transform = TestBaseTransform((104, 117, 123))

  def detect(img, min_score):
    thresh = min_score
    max_im_shrink = ( (max_size*max_size) / (img.shape[0] * img.shape[1])) ** 0.5
    shrink = max_im_shrink if max_im_shrink < 1 else 1

    det0 = infer(net, img, transform, thresh, shrink)
    det1 = infer_flip(net , img , transform , thresh , shrink)
    # shrink detecting and shrink only detect big face
    st = 0.5 if max_im_shrink >= 0.75 else 0.5 * max_im_shrink
    det_s = infer(net , img , transform , thresh , st)
    index = np.where(np.maximum(det_s[:, 2] - det_s[:, 0] + 1, det_s[:, 3] - det_s[:, 1] + 1) > 30)[0]
    det_s = det_s[index, :]
    # enlarge one times
    factor = 2
    bt = min(factor, max_im_shrink) if max_im_shrink > 1 else (st + max_im_shrink) / 2
    det_b = infer(net , img , transform , thresh , bt)
    # enlarge small iamge x times for small face
    if max_im_shrink > factor:
        bt *= factor
        while bt < max_im_shrink:
            det_b = np.row_stack((det_b, infer(net , img , transform , thresh , bt)))
            bt *= factor
        det_b = np.row_stack((det_b, infer(net , img , transform , thresh , max_im_shrink) ))
    # enlarge only detect small face
    if bt > 1:
        index = np.where(np.minimum(det_b[:, 2] - det_b[:, 0] + 1, det_b[:, 3] - det_b[:, 1] + 1) < 100)[0]
        det_b = det_b[index, :]
    else:
        index = np.where(np.maximum(det_b[:, 2] - det_b[:, 0] + 1, det_b[:, 3] - det_b[:, 1] + 1) > 30)[0]
        det_b = det_b[index, :]
    det = np.row_stack((det0, det1, det_s, det_b))
    det = bbox_vote(det)
    out_faces = []
    img_size = img.shape[0]
    for face in det:
      if (face[4] > min_score):
        x0, y0, x1, y1 = face.astype(np.int)[0:4] / img_size
        out_faces.append([x0, y0, x1-x0, y1-y0])
    return out_faces
  return detect

# SSD Mobilenet V1
https://github.com/yeephycho/tensorflow-face-detection

In [0]:
DataDownloader().drive.CreateFile({ 'id': '0B5ttP5kO_loUdWZWZVVrN2VmWFk' }).GetContentFile('./ssd_mobilenet_v1.pb')

In [0]:
import numpy as np
import tensorflow as tf

def create_ssd_mobilenet_v1():
  tf.reset_default_graph()
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  config.allow_soft_placement = True
  config.log_device_placement = True
  detection_graph = tf.Graph()
  with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile('./ssd_mobilenet_v1.pb', 'rb') as fid:
      serialized_graph = fid.read()
      od_graph_def.ParseFromString(serialized_graph)
      tf.import_graph_def(od_graph_def, name='')


  image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
  boxes_tensor = detection_graph.get_tensor_by_name('detection_boxes:0')
  scores_tensor = detection_graph.get_tensor_by_name('detection_scores:0')
  def detect(img, min_score):
    with detection_graph.as_default():
      with tf.Session(graph=detection_graph, config=config) as sess:
        with tf.device('/gpu:0'):
          img = np.expand_dims(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), axis=0)
          (boxes, scores) = sess.run([boxes_tensor, scores_tensor], feed_dict={image_tensor: img})

          indices = np.where(scores > min_score)
          filtered_boxes = boxes[indices][:]
      
          out_boxes = []
          for y0, x0, y1, x1 in filtered_boxes:
            out_boxes.append(x0, y0, x1-x0, y1-y0)
          return out_boxes
  return detect

# Test

## Forward

In [0]:
import numpy as np
from time import time

dbs = ['celeba', 'fddb', 'wider']
min_score = 0.5
min_image_size = 128
max_image_size =  608
#detect = create_retina_face(scales = [1.0], do_flip = False)
#config_name = "retina_scales_1.0_flip_no"
#detect = create_ssd_mobilenet_v1()
#config_name = "ssd_mobilenet_v1"
detect = create_dsfd(608)
config_name = "dsfd_608"
  
for db in dbs:
  for image_size in range(min_image_size, max_image_size + 1, 32):
    print(db, image_size)
    out_filename = "{}_{}_{}".format(config_name, db, image_size)

    test_data = load_json("./{}_testData.json".format(db))
    data_loader = DataLoader(test_data, is_test = True)
    get_next_batch = lambda : data_loader.next_batch(1, image_size)

    next_batch = get_next_batch()
    results = []
    avg_time = 0
    while (next_batch != None):
      batch_x, batch_gt_boxes = next_batch
      gt_boxes = batch_gt_boxes[0]
      ts = time()
      boxes = detect(batch_x[0], min_score)
      avg_time += ((time() - ts) * 1000)
      results.append((boxes, gt_boxes))
      next_batch = get_next_batch()
    avg_time = avg_time / len(results)
    print(avg_time)
    np.save(out_filename, results, allow_pickle = True)

## Test

In [0]:
import numpy as np
from colabsnippets.face_detection import calculate_iou

min_image_size = 128
max_image_size =  608
#config_name = "retina_scales_1.0_flip_no"
config_name = "ssd_mobilenet_v1"
db = "celeba"
iou_thresh = 0.5

image_sizes = []
true_positives_ratios = []
false_positives_ratios = []
for image_size in range(min_image_size, max_image_size + 1, 32):
  filename = "{}_{}_{}.npy".format(config_name, db, image_size)
  try:
    results = np.load(filename, allow_pickle = True)
  except:
    image_sizes.append(image_size)
    true_positives_ratios.append(-1.111)
    false_positives_ratios.append(-1.111)
    continue

  #TODO NMS

  true_positives = 0
  false_positives = 0
  total_boxes = 0
  for res in results:
    pred_boxes, gt_boxes = res
    pred_boxes = np.array(pred_boxes) * image_size
    gt_boxes = np.array(gt_boxes) * image_size
    total_boxes += len(gt_boxes)

    num_preds = len(pred_boxes)
    matched_preds = np.zeros(num_preds, dtype = bool)
    for gt_box in gt_boxes:
      is_detected = False
      for pred_idx, pred_box in enumerate(pred_boxes):
        if (matched_preds[pred_idx]):
          continue
          
        iou = calculate_iou(pred_box, gt_box)
        if iou > iou_thresh:
          matched_preds[pred_idx] = True
          break

    tp = np.sum(matched_preds)
    fp = num_preds - tp
    #print(len(gt_boxes), num_preds, tp, fp)
    true_positives += tp
    false_positives += fp
        
  true_positives_ratio = true_positives / total_boxes
  false_positives_ratio = false_positives / total_boxes
        
  #print(filename)
  #print("- true_positives: {}".format(true_positives))
  #print("- false_positives: {}".format(false_positives))
  #print("- true_positives_ratio: {}".format(true_positives_ratio))
  #print("- false_positives_ratio: {}".format(false_positives_ratio))
  
  image_sizes.append(image_size)
  true_positives_ratios.append(float("{:.4f}".format(true_positives_ratio)))
  false_positives_ratios.append(float("{:.4f}".format(false_positives_ratio)))

out_name = "{}_{}_iou_{}".format(config_name, db, iou_thresh)
print(image_sizes)
print("\"{}\": {{".format(out_name))
print("  \"tp\": {},".format(true_positives_ratios))
print("  \"fp\": {}".format(false_positives_ratios))
print("}},".format(out_name))

# Results

In [0]:
[128, 160, 192, 224, 256, 288, 320, 352, 384, 416, 448, 480, 512, 544, 576, 608]
res_iou_0_5 = {
  "retina_scales_1.0_flip_no_celeba_iou_0.5": {
    "tp": [0.9267, 0.9608, 0.9795, 0.9852, 0.9890, 0.9905, 0.9918, 0.9928, 0.9925, 0.9936, 0.9932, 0.9935, 0.9926, 0.9931, 0.9934, 0.9933],
    "fp": [0.0192, 0.0226, 0.0271, 0.0325, 0.0377, 0.0411, 0.0452, 0.0478, 0.0531, 0.0548, 0.0590, 0.0608, 0.0631, 0.0660, 0.0680, 0.0683]
  },
  "retina_scales_1.0_flip_no_fddb_iou_0.5": {
    "tp": [0.7480, 0.8122, 0.8517, 0.8731, 0.8878, 0.8994, 0.9068, 0.9151, 0.9197, 0.9240, 0.9290, 0.9313, 0.9344, 0.9339, 0.9329, 0.9341],
    "fp": [0.0180, 0.0199, 0.0209, 0.0286, 0.0325, 0.0385, 0.0418, 0.0418, 0.0462, 0.0468, 0.0478, 0.0505, 0.0518, 0.0590, 0.0590, 0.0594]
  },
  "retina_scales_1.0_flip_no_wider_iou_0.5": {
    "tp": [0.0237, 0.0367, 0.0532, 0.0713, 0.0915, 0.1102, 0.1311, 0.1513, 0.1726, 0.1914, 0.2097, 0.2250, 0.2430, 0.2564, 0.2697, 0.2837],
    "fp": [0.0002, 0.0004, 0.0012, 0.0017, 0.0025, 0.0034, 0.0045, 0.0054, 0.0065, 0.0071, 0.0079, 0.0095, 0.0103, 0.0116, 0.0121, 0.0139]
  },
  "ssd_mobilenet_v1_fddb_iou_0.5": {
    "tp": [-1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, 0.7786, 0.7799, 0.7805, 0.7861, 0.7821, 0.7844, 0.7861],
    "fp": [-1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, 0.0234, 0.0232, 0.0236, 0.0248, 0.0248, 0.0242, 0.0224]
  },
  "ssd_mobilenet_v1_wider_iou_0.5": {
    "tp": [-1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, 0.1677, 0.1676, 0.1679, 0.168, 0.1693, 0.1697, 0.1694],
    "fp": [-1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, -1.111, 0.0116, 0.012, 0.0116, 0.0126, 0.0118, 0.0118, 0.0121]
  },
  "tiny_yolo_xception_multiscale_blocks8_128_608_epoch4_fddb_iou_0.5_tp": {
    "tp": [0.3763, 0.4365, 0.4798, 0.4982, 0.4982, 0.5026, 0.4976, 0.5020, 0.4991, 0.5030, 0.5078, 0.5134, 0.5134, 0.5192, 0.5190, 0.5154],
    "fp": [0.2887, 0.1887, 0.1532, 0.1158, 0.1261, 0.1228, 0.1274, 0.1360, 0.1342, 0.1309, 0.1325, 0.1321, 0.1292, 0.1425, 0.1414, 0.1371]
  },
  "tiny_yolo_xception_multiscale_blocks4_128_608_epoch3_fddb_iou_0.5_tp" {
    "tp": [0.3796, 0.4367, 0.4726, 0.5013, 0.5190, 0.5160, 0.5183, 0.5287, 0.5252, 0.5272, 0.5218, 0.5258, 0.5204, 0.5123, 0.5053, 0.4941],
    "fp": [0.3293, 0.2442, 0.2094, 0.1690, 0.1671, 0.1752, 0.1775, 0.1824, 0.1721, 0.1706, 0.1690, 0.1663, 0.1735, 0.1764, 0.1797, 0.1783]
  },
  "tiny_yolo_xception_multiscale_blocks4_128_608_epoch3_wider_iou_0.5_tp" {
    "tp": [0.0177, 0.0222, 0.0238, 0.0243, 0.0269, 0.0256, 0.0256, 0.0267, 0.0274, 0.0278, 0.0276, 0.0279, 0.0282, 0.028, 0.0271, 0.0264],
    "fp": [0.0663, 0.0617, 0.0511, 0.0434, 0.0373, 0.0325, 0.0288, 0.0257, 0.0246, 0.0204, 0.0184, 0.0162, 0.0152, 0.0145, 0.014, 0.0127]
  },
}

# Plot Boxes

In [0]:
from IPython.display import Image, display
from augment.augment import abs_coords
import numpy as np

def draw_box(img, box, color = (255, 0, 0)):
  x, y, w, h = abs_coords(box, img)

  cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
  cv2.circle(img, (x, y), 2, (0, 0, 255), -1)
  cv2.circle(img, (x, y + h), 2, (0, 0, 255), -1)
  cv2.circle(img, (x + w, y), 2, (0, 0, 255), -1)
  cv2.circle(img, (x + w, y + h), 2, (0, 0, 255), -1)
  
!rm -rf ./check_inputs && mkdir ./check_inputs

config_name = "retina_scales_1.0_flip_no"
#config_name = "ssd_mobilenet_v1"
db = "fddb"
image_size = 608
display_size = 608
start_idx = 0
num_images = 10


filename = "{}_{}_{}.npy".format(config_name, db, image_size)
results = np.load(filename, allow_pickle = True)[start_idx : start_idx + num_images]
test_data = load_json("./{}_testData.json".format(db))[start_idx : start_idx + num_images]
images, _ = DataLoader(test_data, is_test = True).next_batch(num_images, display_size)

for idx, img in enumerate(images):
  pred_boxes, gt_boxes = results[idx]
  for box in pred_boxes:
    draw_box(img, box, color = (255, 0, 0))
  for box in gt_boxes:
    draw_box(img, box, color = (0, 255, 0))
  file = './check_inputs/' + str(idx) + '.jpg'
  cv2.imwrite(file, img)
  display(Image(file))
    
!rm -rf ./check_inputs